<a href="https://colab.research.google.com/github/bogatovam/cv-hse/blob/main/CV_HW_18_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подготовка окружения

In [1]:
import cv2
import numpy as np

## Задание №1

Первая камера повернута относительно второй на 45 градусов по оси z и сдвинута на 10 по оси x (в системе отсчета, связанной с первой камерой). У обоих камер матрицы внутренних параметров единичные. Найти фундаментальную матрицу.

**Решение.** 
Формула фундаментальной матрицы:
>$F = K^{-T}[T]RK^{-1} = / K  = E / =[T]_XR$

$[T]_X$ представим как кососимметричную матрицу


In [2]:
angle = np.deg2rad(45)

R =  [[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]
R = np.array(R)

T = [[0, 0,   0],
     [0, 0, -10],
     [0, 10,  0]]

T = np.array(T)

In [3]:
F = np.matmul(T, R)
print("Fundamental matrix:\n{}".format(F))

Fundamental matrix:
[[  0.           0.           0.        ]
 [  0.           0.         -10.        ]
 [  7.07106781   7.07106781   0.        ]]


## Задание №2

Первая камера повернута относительно мировой системы координат на 45 градусов по оси z, а вторая – на -45 градусов по оси у и сдвинута на 10 по оси x. У обоих камер матрицы внутренних параметров единичные. Найти фундаментальную матрицу

**Решение.** 

Матрица проекции первой камеры:
>$P_1 = [R_1|T_1]$

Матрица проекции второй камеры:
>$P_2 = [R_2|T_2]$

Формула нахождения фундаментальной матрицы через матрицы проекций:
>$F = [e']_X P_1 {P_2}^+$

где $e'= P_2O$  и  $O_1 = (0,0,0) + T_1$ 


In [4]:
angle = np.deg2rad(45)

R_1 =[[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]

T_1 = [[0, 0, 0]]

T_1 = np.array(T_1).T

P_1 = np.concatenate((R_1, T_1), axis=1)

In [5]:
angle_2 = np.deg2rad(-45)

R_2 =[[np.cos(angle_2),   0,   np.sin(angle_2)],
      [0,                 1,                 0],
      [-np.sin(angle_2),  0,   np.cos(angle_2)]]

T_2 = [[10, 0, 0]]

T_2 = np.array(T_2).T

P_2 = np.concatenate((R_2, T_2), axis=1)

In [6]:
P_1_pinv = np.linalg.pinv(P_1)

In [7]:
O_1 = np.array([[0, 0, 0]]).T + T_1
e_2 = P_2.dot(np.append(O_1, np.array([1])))

In [8]:
e_2 = [[ 0,      -e_2[2],   e_2[1]  ],
       [ e_2[2],       0,   -e_2[0] ],
       [-e_2[1],  e_2[0],        0] ]

e_2 = np.array(e_2)

In [9]:
F = e_2.dot(P_2).dot(P_1_pinv)

In [10]:
print("Fundamental matrix:\n{}".format(F))

Fundamental matrix:
[[ 0.          0.          0.        ]
 [-5.         -5.         -7.07106781]
 [-7.07106781  7.07106781  0.        ]]


## Задание №3

В задаче 1 найти оба эпиполя

**Решение.** 
> $e_1= P_1O_2$  и  $O_2 = (0,0,0) + T_2 = T_2$ 

> $e_2= P_2O_1$  и  $O_1 = (0,0,0) + T_1$ 

In [11]:
T_1 = np.array([[10, 0, 0]]).T
T_2 = np.array([[0, 0, 0]]).T

O_1 = np.array([[0, 0, 0]]).T + T_1
O_2 = np.array([[0, 0, 0]]).T + T_2

In [12]:
angle = np.deg2rad(45)

R_1 =[[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]

P_1 = np.concatenate((R_1, T_1), axis=1)

In [13]:
R_2 = np.diag(np.ones((3, )))
P_2 = np.concatenate((R_2, T_2), axis=1)

In [14]:
e_1 = P_1.dot(np.append(O_2, np.array([1])))
e_2 = P_2.dot(np.append(O_1, np.array([1])))

In [15]:
print("Epipolar line #1:\n {}".format(e_1))
print("Epipolar line #2:\n {}".format(e_2))

Epipolar line #1:
 [10.  0.  0.]
Epipolar line #2:
 [10.  0.  0.]


## Задание №4

В задаче 1 найти эпиполярную линию, проходящую через точку (0,0) на первом изображении, и соответствующую ей линию на втором изображении

In [16]:
angle = np.deg2rad(45)

R =  [[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]
R = np.array(R)

T = [[0, 0,   0],
     [0, 0, -10],
     [0, 10,  0]]

T = np.array(T)
F = np.matmul(T, R)

In [17]:
q_1 = np.array([[0, 0, 1]]).T
l_2 = F.dot(q_1).T

In [18]:
q_2 = P_2.dot(np.linalg.pinv(P_1)).dot(q_1)

In [19]:
l_1 = q_2.T.dot(F)

In [20]:
print("Epipolar line #1:\n {}".format(l_1))

Epipolar line #1:
 [[7.07106781 7.07106781 0.        ]]


In [21]:
print("Epipolar line #2:\n {}".format(l_2))

Epipolar line #2:
 [[  0. -10.   0.]]


## Задание №5

Вторая камера сдвинута относительно первой вдоль оси x (в системе отсчета, связанной с первой камерой). Найти эпиполярную линию на изображении второй камеры, соответствующую эпиполярной линии на изображении первой
камеры, заданной вектором (0,1,0). 

**Решение.** 
>$l_2 = Fq_1$

>$l_1 = q_2^TF  $

>$q_2^TFq_1 = 0 \rightarrow  l_1q_1 = 0 $

Находим $q_1$ и подставляем его в первую формулу. Посчитаем вручную:

$\begin{pmatrix} 0\\1\\0\end{pmatrix} *\begin{pmatrix}x & y & z\end{pmatrix} = 0$

$\rightarrow q_1 = \begin{pmatrix}x & y & z\end{pmatrix} = \begin{pmatrix}a & 0 & b\end{pmatrix}$

Пусть $q_1 = \begin{pmatrix}1 & 0 & 1\end{pmatrix}$

Далее посчитаем $l_2 = Fq_1$

In [24]:
F = [[0, 0,   0],
     [0, 0, -1],
     [0, 1,  0]]

F = np.array(F)

q_1 = np.array([[1, 0, 1]]).T

print("Epipolar line:\n {}".format(F.dot(q_1)))

Epipolar line:
 [[ 0]
 [-1]
 [ 0]]
